# Rekordbox Duplicate Track Removal-Inator

Given a Rekordbox collection XML file, this code finds duplicate (Artist, Track Name, Album) entries and identifies which duplicates to remove.

Where one or more duplicate entries have cuepoints, this program scores each entry according to the number of cuepoints and their type. This score is designed to weigh loops more than basic cues.

The goal is to clean up my Rekordbox collection, which, for reasons I don't know, is full of duplicates.

The following code was tested on XML output from Rekordbox version 6.8.3.

## Load useful libraries

In [1]:
import xmltodict

The following is part of the toolkit I'm writing to deal with Rekordbox issues:

In [2]:
from rekordbox_interface import *

## User settings

In [3]:
xml_file = '../xml.xml'

## Load the XML exported from rekordbox and parse into a Python dictionary

In [4]:
with open(xml_file, 'r') as file:
    xml_as_string = file.read().replace('\n', '')

In [5]:
dict_rekordbox_content = xmltodict.parse(xml_as_string)

In [6]:
class_all_tracks = rkb_all_tracks(dict_rekordbox_content)

## Analyze duplicate entries in Rekordbox database to decide which of a duplicate pair to keep

#### Identify the duplicates

In [7]:
def identify_duplicate_track_entries(tracks):

    df_agg_to_find_duplicates = (
        tracks
        .df_track_info
        .groupby(['Name', 'Artist', 'Album'])
        .agg('count')
        ['TrackID']
        .reset_index()
        .rename(columns={'TrackID' : 'count'})
    )

    df_duplicates = (
        pd.merge(
            df_agg_to_find_duplicates[df_agg_to_find_duplicates['count'] > 1],
            tracks.df_track_info,
            on = ['Name', 'Artist', 'Album'],
            how = 'left',
        )[['Artist', 'Name', 'Album', 'TrackID']]
        .sort_values(by = ['Artist', 'Name', 'Album', 'TrackID'])
        .copy()
    )
    
    return(df_duplicates)

In [8]:
df_duplicates = identify_duplicate_track_entries(class_all_tracks)

df_duplicates

,Artist,Name,Album,TrackID
194,,HORN,,94849455
195,,HORN,,244339207
530,2 Indabush,The Lick,Club Flys 3-Late Night,39578582
531,2 Indabush,The Lick,Club Flys 3-Late Night,232360351
44,8.02,Believe (Extended Mix),Believe,46698218
...,...,...,...,...
450,Waldhaus,Set Me Free (Original Mix),Set Me Free,169662044
253,Zsa Zsa Deluxe,Inside of Me (Cs Johansen Remix),Best of Rave Classics (Remix Edition) [Explicit],250715880
252,Zsa Zsa Deluxe,Inside of Me (Cs Johansen Remix),Best of Rave Classics (Remix Edition) [Explicit],256757927
112,kWASP,Distorted People (Original Mix),Distorted People,132769205


#### Where duplicate tracks are found, select which one to keep

By first deciding which to keep, we avoid any risk in case there are more than two duplicates for a given track:

In [9]:
def identify_duplicate_tracks_to_keep(df_duplicates, tracks):

    #
    # compute the components necessary for computing a score for each duplicated track
    #
    df_score = (
        pd.merge(
            df_duplicates,
            tracks.df_all_cue_positions,
            on = ['TrackID'],
            how = 'left',
        )
        .fillna({'Type' : 0.})
        .rename(columns = {'Name_x' : 'track_name', 'Name_y' : 'cue_name'})
        .groupby(['Artist', 'track_name', 'Album', 'TrackID'])
        ['Type']
        .agg(['sum', 'count'])
        .reset_index()
    )

    #
    # compute score
    # 
    # then convert resulting score to integer (I think this will produce more stable joins
    # than floating values for score would)
    #
    # for some reason, this works better with "int(x)" rather than "np.int64(...)"
    #
    df_score['score'] = (df_score['sum'] + df_score['count']) / 2
    df_score['score'] = [int(x) for x in df_score['score']]

    #
    # determine which of a given duplicate track carries the highest score
    # (we want to keep these)
    #
    df_max_score = (
        df_score
        .groupby(['Artist', 'track_name', 'Album'])
        ['score']
        .agg('max')
        .reset_index()
    )

    #
    # convert resulting score to integer (I think this will produce more stable joins
    # than floating values for score would)
    #
    # for some reason, this works better with "int(x)" rather than "np.int64(...)"
    #
    df_max_score['score'] = [int(x) for x in df_max_score['score']]

    #
    # join to retrieve TrackID
    #
    df_tracks_score = (
        pd.merge(
            df_max_score,
            df_score[['Artist', 'track_name', 'Album', 'score', 'TrackID']],
            on = ['Artist', 'track_name', 'Album', 'score'],
            how = 'left'
        )
    )

    #
    # if there are multiple rows for a given ('Artist', 'track_name', 'Album', 'score')
    # having the same max score, we keep the one with the lowest 'TrackID', assuming
    # it was the earliest loaded
    #
    df_earliest = (
        df_tracks_score
        .groupby(['Artist', 'track_name', 'Album'])
        ['TrackID']
        .agg('min')
        .reset_index()
    )
    
    df_earliest['keep'] = True
    
    return df_earliest

In [10]:
df_earliest = identify_duplicate_tracks_to_keep(df_duplicates, class_all_tracks)

df_earliest[['Artist', 'track_name', 'Album', 'TrackID']]

,Artist,track_name,Album,TrackID
0,,HORN,,94849455
1,2 Indabush,The Lick,Club Flys 3-Late Night,39578582
2,8.02,Believe (Extended Mix),Believe,46698218
3,Aannaa,OINGO BOINGO_WEIRD SCIENCE_AANNAA REMIX,Lockup,3607748
4,Alannah Myles,Black Velvet,Striptease Music,10843849
...,...,...,...,...
306,Vareso,Come With Me (Extended Mix),Come With Me,132818211
307,Vincent Caira,This Clean (Original Mix),This Clean,19126218
308,Waldhaus,Set Me Free (Original Mix),Set Me Free,87882711
309,Zsa Zsa Deluxe,Inside of Me (Cs Johansen Remix),Best of Rave Classics (Remix Edition) [Explicit],256757927


#### Figure out the tracks to remove

Because we select only one to keep (in case there exists more than one duplicate), we can now robustly identify which tracks to remove:

In [11]:
def figure_out_which_tracks_to_remove(df_duplicates, df_earliest):
    df_all_duplicated_tracks = (
        pd.merge(
            df_duplicates,
            df_earliest[['TrackID', 'keep']],
            on = 'TrackID',
            how = 'left',
        )
        .fillna({'keep' : False})
    )
    df_false = df_all_duplicated_tracks[~df_all_duplicated_tracks['keep']]
    
    return sorted(list(df_false['TrackID'].values))

## We'll use these results later to reconstruct the XML without the duplicates

I'll write that code later, after an EXTENSIVE backup of my current track library.

In [12]:
df_tracks_to_remove = figure_out_which_tracks_to_remove(df_duplicates, df_earliest)

df_tracks_to_remove

[3429062,
 12701254,
 16811311,
 25261442,
 29388175,
 30655891,
 30891963,
 30925493,
 32384172,
 34368550,
 39723179,
 40017280,
 40508267,
 43228081,
 43484650,
 44525925,
 49431154,
 49520596,
 54907449,
 54954678,
 57261431,
 58079814,
 58298479,
 60052045,
 60392567,
 61083825,
 61411077,
 63575365,
 64927631,
 66065039,
 66340879,
 68414878,
 69675834,
 69738312,
 70851859,
 72505140,
 73357303,
 74132723,
 75376485,
 76200870,
 77774510,
 78997152,
 83447217,
 85134900,
 85252860,
 85590321,
 86117985,
 87518576,
 89418452,
 90597190,
 90758377,
 91290961,
 92682568,
 92907093,
 92971575,
 93751960,
 93973548,
 94359416,
 94693555,
 94748269,
 96063061,
 99726780,
 101292769,
 103617769,
 103695715,
 104338978,
 104776060,
 107104587,
 107117042,
 107811460,
 108085663,
 110819995,
 112053744,
 112167172,
 113057562,
 114815813,
 114930045,
 115925832,
 116723211,
 117955149,
 118576448,
 120872668,
 121825358,
 122281636,
 122401079,
 122422886,
 122528659,
 123686181,
 124552